In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import tensorflow as tf

In [2]:
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = False
#configuration.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.compat.v1.Session(config=configuration)


In [3]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [4]:
env = ShowerEnv()

D:\programas\anaco\envs\Gpu-tf25\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [5]:
env.observation_space.sample()

array([67.52203], dtype=float32)

In [6]:
with tf.device('/gpu:0'):
    episodes = 10
    for episode in range(1, episodes+1):
        state = env.reset()
        done = False
        score = 0 
    
        while not done:
            #env.render()
            action = env.action_space.sample()
            n_state, reward, done, info = env.step(action)
            score+=reward
        print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-36
Episode:2 Score:-60
Episode:3 Score:-30
Episode:4 Score:-34
Episode:5 Score:-30
Episode:6 Score:-30
Episode:7 Score:-12
Episode:8 Score:14
Episode:9 Score:-46
Episode:10 Score:-26


In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow

In [8]:
states = env.observation_space.shape
actions = env.action_space.n

In [9]:
actions

3

In [10]:
def build_model(states, actions):
    with tf.device('/gpu:0'):
        model = tensorflow.keras.models.Sequential()   
        model.add(tensorflow.keras.layers.Dense(24, activation='relu', input_shape=states))
        model.add(tensorflow.keras.layers.Dense(24, activation='relu'))
        model.add(tensorflow.keras.layers.Dense(actions, activation='linear'))
        return model

In [11]:

#with tf.device('/gpu:0'):
model = build_model(states, actions)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [13]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [14]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [15]:
with tf.device('/gpu:0'):
    dqn = build_agent(model, actions)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

D:\programas\anaco\envs\Gpu-tf25\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


AttributeError: 'Sequential' object has no attribute '_compile_time_distribution_strategy'